In [1]:
%pylab inline
import warnings
warnings.filterwarnings("ignore")
import nolearn
from nolearn.lasagne import NeuralNet
import readdata
import lasagne
from lasagne import layers
from sklearn import metrics
import detectobjects as det
import os.path
from scipy import misc
import cv2
from progress_bar import ProgressBar
import shapefeatures
from sklearn import ensemble
from PIL import Image

Populating the interactive namespace from numpy and matplotlib


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import readdata

In [3]:
opts = {'img_dir': './plasmodium-phonecamera/images/',
        'annotation_dir': './plasmodium-phonecamera/annotation/',
        'detection_probability_threshold': 0.5,
        'detection_overlap_threshold': 0.3, 
        'gauss': 1,
        'patch_size': (50,50),
        'image_downsample' : 2,
        'detection_step': 5,
        'patch_creation_step': 40,
        'object_class': None,
        'negative_training_discard_rate': .9
       }
opts['patch_stride_training'] = int(opts['patch_size'][0]*.25)

In [4]:
def convert_rgb_togray(img_array):
    img = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    return img

In [5]:
def get_pixel_values(img_array):
    img = Image.fromarray(img_array)
    return list(img.getdata())

In [6]:
def mse(imageA,imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
#     print(imageA.shape)
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [7]:
def mse1(imageA,imageB):
    # the 'Mean Squared Error' between the two images is the
    # sum of the squared difference between the two images;
    # NOTE: the two images must have the same dimension
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    # return the MSE, the lower the error, the more "similar"
    # the two images are
    return err

In [8]:
"""Function takes in numpy array of all positive patches and calculates 
the mean along the vertical axis
"""
def mean_pos_patch(patches):
    p = np.array(patches).astype(np.float)
    return np.mean(p,axis=0)

In [21]:
patchset_image = "./plasmodium-phonecamera/pth_image/"
patchset_annotation = "./plasmodium-phonecamera/pth_anno/"

pos_patchset_save_pth = "./plasmodium-phonecamera/pos/"
neg_patchset_save_pth = "./plasmodium-phonecamera/neg/"


# trainfiles, valfiles, testfiles = readdata.create_sets(patchset_image, train_set_proportion=.125, 
#                                                   test_set_proportion=.25,
#                                                   val_set_proportion=0)

# train_y, train_X = readdata.create_patches(trainfiles, patchset_annotation, patchset_image, opts['patch_size'][0], opts['patch_stride_training'], grayscale=False, progressbar=True, downsample=opts['image_downsample'], objectclass=opts['object_class'], negative_discard_rate=opts['negative_training_discard_rate'])
# test_y, test_X = readdata.create_patches(testfiles,  patchset_annotation, patchset_image, opts['patch_size'][0], opts['patch_stride_training'], grayscale=False, progressbar=True, downsample=opts['image_downsample'], objectclass=opts['object_class'], negative_discard_rate=opts['negative_training_discard_rate'])

# Cut down on dispropor
trainfiles, valfiles, testfiles = readdata.create_sets(opts['img_dir'], train_set_proportion=.5, 
                                                  test_set_proportion=.5,
                                                  val_set_proportion=0)


# print(len(trainfiles))

# train_y, train_X = readdata.create_patches(t, patchset_annotation, patchset_image, opts['patch_size'][0], opts['patch_stride_training'], grayscale=False, progressbar=True, downsample=opts['image_downsample'], objectclass=opts['object_class'], negative_discard_rate=opts['negative_training_discard_rate'])
sum_false_pos = 0
sum_neg_pixel = 0

for t in trainfiles:
    train_y, train_X = readdata.create_patches_per_image(t, opts['annotation_dir'], opts['img_dir'], opts['patch_size'][0], opts['patch_stride_training'], grayscale=False, progressbar=True, downsample=opts['image_downsample'], objectclass=opts['object_class'], negative_discard_rate=opts['negative_training_discard_rate'])

# Re-order numpy array to (size,size,channels)
    train_X =  np.swapaxes(train_X,1,3)

# Split positive and negative pacthes
    pos_pixel = [p for p,c in zip(train_X, train_y) if c==1]
    neg_pixel = [p for p,c in zip(train_X, train_y) if c==0]
    
    
# Number of positive and negative patches for each image
    print("\n")
    print("Positive patches:", len(pos_pixel))
    print("Negative patches:", len(neg_pixel))
    sum_neg_pixel+=len(neg_pixel)
    print("End of patch count\n\n")
    
    if len(pos_pixel)>0:
    # Get avg of positive patches
        positive_patch_ref = mean_pos_patch(pos_pixel)

    # Calculate mse of negative patches with reference to avg positive patch
        mse_values = [mse1(positive_patch_ref,n) for n in neg_pixel]
        print("Sample mse values:", mse_values[0:5])

    # Get positions of falsen positives in negative patch array
        false_pos_indices = [mse_values.index(i) for i in mse_values if i <= np.mean(mse_values)]
        
    # Get false positive patches
        false_pos_patches = [neg_pixel[f] for f in false_pos_indices]
        print("False_positives", len(false_pos_patches))
        
        sum_false_pos+=len(false_pos_patches)
        

#print sum_false_pos
#print sum_neg_pixel
print("\n")
print("Total number of false_Positive patches:", sum_false_pos)
print("Total number of Negative patches:", sum_neg_pixel)

        
    
# mse_values = []
# positive_patch_ref = mean_pos_patch(pos_pixel)
# for n in neg_pixel:
#     mse_values.append(mse1(positive_patch_ref,n))
# print(mse_values[0:5])


[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 370)
End of patch count


[                  0%                  ] 

('Positive patches:', 2)
('Negative patches:', 358)
End of patch count


('Sample mse values:', [265.7276, 637.9596, 299.8524, 456.5212, 238.3276])
('False_positives', 228)
[                  0%                  ] 

('Positive patches:', 10)
('Negative patches:', 308)
End of patch count


('Sample mse values:', [497.75092800000004, 966.8286879999999, 281.249488, 758.461328, 843.584208])
('False_positives', 285)
[                  0%                  ] 

('Positive patches:', 15)
('Negative patches:', 275)
End of patch count


('Sample mse values:', [983.8710471111111, 607.8653937777779, 791.6321137777777, 1397.5633137777775, 1364.4643271111113])
('False_positives', 232)
[                  0%                  ] 

('Positive patches:', 3)
('Negative patches:', 324)
End of patch count


('Sample mse values:', [1295.4387555555556, 45

[                  0%                  ] 

('Positive patches:', 5)
('Negative patches:', 315)
End of patch count


('Sample mse values:', [486.88377599999995, 393.336256, 397.571776, 24705.195456, 555.200576])
('False_positives', 261)
[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 345)
End of patch count


[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 350)
End of patch count


[                  0%                  ] 

('Positive patches:', 15)
('Negative patches:', 287)
End of patch count


('Sample mse values:', [332.8909937777778, 376.8048071111111, 259.47158044444444, 175.5300337777778, 853.7596871111111])
('False_positives', 262)
[                  0%                  ] 

('Positive patches:', 18)
('Negative patches:', 263)
End of patch count


('Sample mse values:', [307.5480049382716, 324.3997382716049, 957.3800049382717, 997.3517382716049, 769.4744049382716])
('False_positives', 240)
[      

[                  0%                  ] 

('Positive patches:', 12)
('Negative patches:', 310)
End of patch count


('Sample mse values:', [1621.5905111111113, 1184.0665111111111, 351.81077777777773, 1653.3958444444445, 394.4977111111111])
('False_positives', 195)
[                  0%                  ] 

('Positive patches:', 14)
('Negative patches:', 265)
End of patch count


('Sample mse values:', [710.3919918367346, 657.2555346938776, 351.95862040816326, 842.0219346938776, 308.626506122449])
('False_positives', 191)
[                  0%                  ] 

('Positive patches:', 9)
('Negative patches:', 298)
End of patch count


('Sample mse values:', [750.0768, 338.70542222222224, 253.86862222222226, 1540.2910222222222, 4992.116444444444])
('False_positives', 218)
[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 369)
End of patch count


[                  0%                  ] 

('Positive patches:', 12)
('Negative patches:', 295)
End o

[                  0%                  ] 

('Positive patches:', 5)
('Negative patches:', 324)
End of patch count


('Sample mse values:', [717.1503359999999, 729.5775359999999, 450.73977599999995, 476.13465599999995, 439.18393599999996])
('False_positives', 297)
[                  0%                  ] 

('Positive patches:', 9)
('Negative patches:', 298)
End of patch count


('Sample mse values:', [463.29471604938266, 887.7647604938272, 1396.7500049382716, 170.15960493827157, 149.09382716049384])
('False_positives', 275)
[                  0%                  ] 

('Positive patches:', 10)
('Negative patches:', 309)
End of patch count


('Sample mse values:', [346.7868, 209.9404, 482.2876, 646.04056, 416.90008])
('False_positives', 251)
[                  0%                  ] 

('Positive patches:', 3)
('Negative patches:', 332)
End of patch count


('Sample mse values:', [450.9688888888889, 679.9864888888889, 737.2046222222222, 355.0366222222222, 618.3790222222221])
('False_positive

('False_positives', 266)
[                  0%                  ] 

('Positive patches:', 6)
('Negative patches:', 316)
End of patch count


('Sample mse values:', [622.3336888888889, 278.6600888888889, 311.1011555555556, 794.847288888889, 778.2723555555556])
('False_positives', 256)
[                  0%                  ] 

('Positive patches:', 8)
('Negative patches:', 303)
End of patch count


('Sample mse values:', [749.499675, 477.498475, 914.332075, 549.443275, 165.450875])
('False_positives', 150)
[                  0%                  ] 

('Positive patches:', 2)
('Negative patches:', 360)
End of patch count


('Sample mse values:', [563.6228, 305.7812, 545.5044, 467.154, 441.0084])
('False_positives', 340)
[                  0%                  ] 

('Positive patches:', 9)
('Negative patches:', 318)
End of patch count


('Sample mse values:', [144.42490864197532, 142.817975308642, 251.45388641975308, 178.19201975308644, 198.33850864197532])
('False_positives', 208)
[         

...

('False_positives', 223)
[                  0%                  ] 

('Positive patches:', 14)
('Negative patches:', 281)
End of patch count


('Sample mse values:', [20107.326081632655, 460.7594530612245, 13350.735224489796, 107.9617387755102, 386.35808163265307])
('False_positives', 244)
[                  0%                  ] 

('Positive patches:', 5)
('Negative patches:', 330)
End of patch count


('Sample mse values:', [558.180416, 1213.564416, 492.46393599999993, 372.356096, 676.9135359999999])
('False_positives', 190)
[                  0%                  ] 

('Positive patches:', 12)
('Negative patches:', 307)
End of patch count


('Sample mse values:', [1025.2975333333331, 1222.9356666666665, 335.61086666666665, 320.3708666666667, 1068.6732666666667])
('False_positives', 265)
[                  0%                  ] 

('Positive patches:', 17)
('Negative patches:', 258)
End of patch count


('Sample mse values:', [1716.4566366782005, 262.56901314878894, 283.8743307958477, 23

[                  0%                  ] 

('Positive patches:', 9)
('Negative patches:', 297)
End of patch count


('Sample mse values:', [954.6994567901235, 203.03403456790127, 508.8870123456789, 721.2139456790123, 514.175901234568])
('False_positives', 150)
[                  0%                  ] 

('Positive patches:', 1)
('Negative patches:', 341)
End of patch count


('Sample mse values:', [1396.3248, 870.032, 1310.232, 721.1664, 1449.2224])
('False_positives', 283)
[                  0%                  ] 

('Positive patches:', 4)
('Negative patches:', 328)
End of patch count


('Sample mse values:', [800.4032, 1276.5152, 1666.3328, 646.3592, 690.5648])
('False_positives', 272)
[                  0%                  ] 

('Positive patches:', 6)
('Negative patches:', 335)
End of patch count


('Sample mse values:', [450.9812444444445, 498.8868444444444, 377.0548444444444, 525.5129777777778, 7260.90551111111])
('False_positives', 289)
[                  0%                  ] 

(

[                  0%                  ] 

('Positive patches:', 2)
('Negative patches:', 340)
End of patch count


('Sample mse values:', [997.9912, 268.3576, 474.5096, 247.7224, 330.4632])
('False_positives', 310)
[                  0%                  ] 

('Positive patches:', 2)
('Negative patches:', 337)
End of patch count


('Sample mse values:', [604.9888, 814.6768, 252.0976, 711.52, 541.2592])
('False_positives', 219)
[                  0%                  ] 

('Positive patches:', 9)
('Negative patches:', 321)
End of patch count


('Sample mse values:', [290.0629135802469, 319.4662913580247, 255.0659358024691, 353.5656691358024, 202.23944691358022])
('False_positives', 274)
[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 347)
End of patch count


[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 371)
End of patch count


[                  0%                  ] 

('Positive patches:', 7)
('Negati

[                  0%                  ] 

('Positive patches:', 3)
('Negative patches:', 359)
End of patch count


('Sample mse values:', [226.4222222222222, 857.2046222222222, 1296.532088888889, 299.1123555555556, 276.3752888888889])
('False_positives', 273)
[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 370)
End of patch count


[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 371)
End of patch count


[                  0%                  ] 

('Positive patches:', 7)
('Negative patches:', 318)
End of patch count


('Sample mse values:', [1757.2595265306122, 2962.787983673469, 596.8458122448978, 2967.303183673469, 226.2533551020408])
('False_positives', 242)
[                  0%                  ] 

('Positive patches:', 0)
('Negative patches:', 346)
End of patch count


[                  0%                  ] 

('Positive patches:', 3)
('Negative patches:', 352)
End of patch count


('Sample mse 

[                  0%                  ] 

('Positive patches:', 2)
('Negative patches:', 360)
End of patch count


('Sample mse values:', [175.6608, 228.0432, 284.9952, 344.2672, 502.6912])
('False_positives', 338)
[                  0%                  ] 

('Positive patches:', 6)
('Negative patches:', 335)
End of patch count


('Sample mse values:', [249.32168888888884, 553.3936888888888, 315.1142222222222, 303.89022222222223, 465.2086222222222])
('False_positives', 238)
[                  0%                  ] 

('Positive patches:', 1)
('Negative patches:', 366)
End of patch count


('Sample mse values:', [402.736, 752.3024, 376.8352, 334.2352, 502.2992])
('False_positives', 196)
[                  0%                  ] 

('Positive patches:', 1)
('Negative patches:', 362)
End of patch count


('Sample mse values:', [1207.7744, 754.872, 672.2528, 910.36, 1203.784])
('False_positives', 199)
[                  0%                  ] 

('Positive patches:', 6)
('Negative patches:', 34